<a href="https://colab.research.google.com/github/antoniogss/air_quality/blob/main/db_covs_pinheiros.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import warnings as war
import google.colab as gc
import matplotlib.pyplot as plt
import os

from datetime import date, time, datetime, timedelta

In [ ]:
gc.drive.mount('./gdrive/', 
               force_remount=True
              )
gc.output.clear()

In [ ]:
%cd ./gdrive/MyDrive/air_quality/datasets/COVs/

/content/gdrive/MyDrive/air_quality/datasets/COVs


In [ ]:
# Leitura de arquivos no formato CSV

'''
Define o caminho onde os arquivos CSV estão armazenados.
'''

caminho = './'

'''
A lista "anos" permite selecionar e processar arquivos específicos com base nos 
anos fornecidos.
'''
ano_inicio = 2017

ano_fim = 2021

anos = list(range(ano_inicio, ano_fim+1))

'''
Usa a biblioteca os para obter uma lista de todos 
os arquivos CSV no caminho especificado e ordená-los 
em ordem alfabética.
'''

arquivos_csv = [f for f in os.listdir(caminho) if f.endswith('.csv')]

'''
A lista "arquivos_selecionados" é composta por arquivos filtrados com base na 
numeração final correspondente aos anos presentes na lista "anos".
'''
arquivos_selecionados = [arquivo for arquivo in arquivos_csv if int(arquivo[-8:-4]) in anos]

'''
Cria uma lista de dataframes, um para cada arquivo CSV 
encontrado, usando a biblioteca pandas . Os dataframes 
são lidos com o separador '|' e o encoding 'utf8'.
'''

lista_dfs = [pd.read_csv(os.path.join(caminho, arquivo), sep=';', encoding='latin-1', skiprows=7) for arquivo in arquivos_selecionados]

'''
Concatena todos os dataframes da lista em um único dataframe, 
usando o método concat() da biblioteca pandas. O argumento 
ignore_index=True é usado para redefinir os índices do 
dataframe resultante.
'''

df_covs = pd.concat(lista_dfs, ignore_index=True)

In [ ]:
df_covs.head()

,Unnamed: 0,Unnamed: 1,BEN(Benzeno) - µg/m3,TOL(Tolueno) - µg/m3
0,01/01/2017,01:00,NaN,NaN
1,01/01/2017,02:00,NaN,NaN
2,01/01/2017,03:00,NaN,NaN
3,01/01/2017,04:00,NaN,NaN
4,01/01/2017,05:00,NaN,NaN


In [ ]:
colunas = {'Unnamed: 0':'data',
           'Unnamed: 1':'hora',
           'BEN(Benzeno) - µg/m3':'benzeno',
           'TOL(Tolueno) - µg/m3':'tolueno'}

df_covs.rename(
               columns=colunas,
               inplace = True
              )

df_covs.head()

,data,hora,benzeno,tolueno
0,01/01/2017,01:00,NaN,NaN
1,01/01/2017,02:00,NaN,NaN
2,01/01/2017,03:00,NaN,NaN
3,01/01/2017,04:00,NaN,NaN
4,01/01/2017,05:00,NaN,NaN


In [ ]:
df_covs = df_covs.replace({',' : '.'}, regex = True)

In [ ]:
df_covs['benzeno'] = pd.to_numeric(df_covs['benzeno'], downcast='float', errors='coerce')
df_covs['tolueno'] = pd.to_numeric(df_covs['tolueno'], downcast='float', errors='coerce')

In [ ]:
df_covs.describe()

,benzeno,tolueno
count,34518.000000,34508.000000
mean,1.042934,5.698737
std,1.342156,6.781441
min,0.000000,0.000000
25%,0.000000,2.000000
50%,1.000000,4.000000
75%,1.000000,7.000000
max,22.000000,98.000000


In [ ]:
df_covs['hora'].replace('24:00', '00:00', inplace = True)
gc.output.clear()

In [ ]:
df_covs['data_hora'] = df_covs['data'] + ' ' + df_covs['hora']
df_covs.head()

,data,hora,benzeno,tolueno,data_hora
0,01/01/2017,01:00,NaN,NaN,01/01/2017 01:00
1,01/01/2017,02:00,NaN,NaN,01/01/2017 02:00
2,01/01/2017,03:00,NaN,NaN,01/01/2017 03:00
3,01/01/2017,04:00,NaN,NaN,01/01/2017 04:00
4,01/01/2017,05:00,NaN,NaN,01/01/2017 05:00


In [ ]:
df_covs = df_covs.drop(
              [
                'data', 
                'hora'
              ],
              axis = 1
)
df_covs.head()

,benzeno,tolueno,data_hora
0,NaN,NaN,01/01/2017 01:00
1,NaN,NaN,01/01/2017 02:00
2,NaN,NaN,01/01/2017 03:00
3,NaN,NaN,01/01/2017 04:00
4,NaN,NaN,01/01/2017 05:00


In [ ]:
df_covs['data_hora'] = pd.to_datetime(df_covs['data_hora'], format = '%d/%m/%Y %H:%M')
df_covs.head()

,benzeno,tolueno,data_hora
0,NaN,NaN,2017-01-01 01:00:00
1,NaN,NaN,2017-01-01 02:00:00
2,NaN,NaN,2017-01-01 03:00:00
3,NaN,NaN,2017-01-01 04:00:00
4,NaN,NaN,2017-01-01 05:00:00


In [ ]:
df_covs.dtypes

benzeno             float32
tolueno             float32
data_hora    datetime64[ns]
dtype: object

In [ ]:
dias_ptbr = {
              'Sunday':'Domingo', 
              'Monday':'Segunda-feira', 
              'Tuesday':'Terça-feira', 
              'Wednesday':'Quarta-feira', 
              'Thursday':'Quinta-feira', 
              'Friday':'Sexta-feira', 
              'Saturday':'Sábado'
}

df_covs['dia_da_semana'] = df_covs['data_hora'].dt.day_name().replace(dias_ptbr)

df_covs.head()

,benzeno,tolueno,data_hora,dia_da_semana
0,NaN,NaN,2017-01-01 01:00:00,Domingo
1,NaN,NaN,2017-01-01 02:00:00,Domingo
2,NaN,NaN,2017-01-01 03:00:00,Domingo
3,NaN,NaN,2017-01-01 04:00:00,Domingo
4,NaN,NaN,2017-01-01 05:00:00,Domingo


In [ ]:
df_covs.to_csv('./df_covs.csv')

In [ ]:
!echo "Total number of lines: df_covs.csv"
!cat df_covs.csv | wc -l

Total number of lines: df_covs.csv
43825
